### Universidad del Valle de Guatemala
#### Brandon Ronaldo Sicay Cumes - 21757
##### Laboratorio 5 - QKD



Chat utilizado https://chatgpt.com/c/679043bd-6990-800f-8b61-e48f1ddf


In [2]:
import random

def simulate_bb84(n_bits=15, include_eve=False):
    """
    Simula el protocolo BB84 para n_bits. Opcionalmente incluye a Eve interceptando.
    
    Parámetros:
        n_bits (int): número de bits a transmitir.
        include_eve (bool): si es True, simula que Eve intercepta cada fotón.
        
    Imprime una tabla con el historial de cada ronda y construye la clave secreta final.
    """
    # Representamos bases como símbolos para mayor claridad:
    BASES = {'rect': '↕', 'diag': '↗'}
    possible_bases = list(BASES.keys())
    
    # Paso 1: Alice elige bits y bases
    alice_bits = [random.randint(0, 1) for _ in range(n_bits)]
    alice_bases = [random.choice(possible_bases) for _ in range(n_bits)]
    
    # Paso 2: Si hay Eve, ella elige bases aleatorias
    if include_eve:
        eve_bases = [random.choice(possible_bases) for _ in range(n_bits)]
    else:
        eve_bases = [None] * n_bits
    
    # Paso 3: Bob elige bases aleatorias
    bob_bases = [random.choice(possible_bases) for _ in range(n_bits)]
    
    # Listas para registrar resultados
    eve_bits = [None] * n_bits
    sent_bits = [None] * n_bits    # Lo que realmente llega a Bob (de Alice o de Eve)
    bob_bits = [None] * n_bits
    bases_match = [False] * n_bits
    use_bit = [False] * n_bits
    
    for i in range(n_bits):
        a_bit = alice_bits[i]
        a_base = alice_bases[i]
        
        # Si Eve intercepta
        if include_eve:
            e_base = eve_bases[i]
            if e_base == a_base:
                e_bit = a_bit
            else:
                e_bit = random.randint(0, 1)
            eve_bits[i] = e_bit
            sent_bits[i] = e_bit
            sent_base = e_base
        else:
            sent_bits[i] = a_bit
            sent_base = a_base
        
        # Bob mide
        b_base = bob_bases[i]
        if b_base == sent_base:
            b_bit = sent_bits[i]
        else:
            b_bit = random.randint(0, 1)
        bob_bits[i] = b_bit
        
        # Comparamos las bases originales de Alice y Bob
        bases_match[i] = (a_base == b_base)
        use_bit[i] = bases_match[i]
    
    # Construir la clave secreta
    secret_key = [str(alice_bits[i]) for i in range(n_bits) if use_bit[i]]
    
    # Imprimir tabla
    header = ["N°", "A_bit", "A_base", "Fotón"]
    if include_eve:
        header += ["E_base", "E_bit"]
    header += ["B_base", "B_bit", "Bases?", "Usar?"]
    print(" | ".join(f"{h:^7}" for h in header))
    print("-" * (9 * len(header)))
    
    for i in range(n_bits):
        row = [
            f"{i+1:^7}",
            f"{alice_bits[i]:^7}",
            f"{BASES[alice_bases[i]]:^7}",
            f"{sent_bits[i]:^7}"
        ]
        if include_eve:
            row += [
                f"{BASES[eve_bases[i]] if eve_bases[i] else ' ':^7}",
                f"{eve_bits[i] if eve_bits[i] is not None else ' ':^7}"
            ]
        row += [
            f"{BASES[bob_bases[i]]:^7}",
            f"{bob_bits[i]:^7}",
            f"{'sí' if bases_match[i] else 'no':^7}",
            f"{'sí' if use_bit[i] else 'no':^7}"
        ]
        print(" | ".join(row))
    
    print("\nClave secreta (bits usados):", "".join(secret_key))
    print(f"Bits obtenidos: {len(secret_key)} de {n_bits} "
          f"({len(secret_key) / n_bits * 100:.1f}%)\n")


if __name__ == "__main__":
    # Simulación sin Eve
    print("=== SIMULACIÓN BB84 SIN EVE ===")
    simulate_bb84(n_bits=15, include_eve=False)
    
    # Simulación con Eve
    print("\n=== SIMULACIÓN BB84 CON EVE ===")
    simulate_bb84(n_bits=15, include_eve=True)


=== SIMULACIÓN BB84 SIN EVE ===
  N°    |  A_bit  | A_base  |  Fotón  | B_base  |  B_bit  | Bases?  |  Usar? 
------------------------------------------------------------------------
   1    |    1    |    ↕    |    1    |    ↕    |    1    |   sí    |   sí   
   2    |    1    |    ↕    |    1    |    ↗    |    0    |   no    |   no   
   3    |    0    |    ↗    |    0    |    ↗    |    0    |   sí    |   sí   
   4    |    0    |    ↗    |    0    |    ↗    |    0    |   sí    |   sí   
   5    |    1    |    ↕    |    1    |    ↕    |    1    |   sí    |   sí   
   6    |    0    |    ↕    |    0    |    ↕    |    0    |   sí    |   sí   
   7    |    1    |    ↗    |    1    |    ↗    |    1    |   sí    |   sí   
   8    |    0    |    ↕    |    0    |    ↕    |    0    |   sí    |   sí   
   9    |    1    |    ↕    |    1    |    ↗    |    0    |   no    |   no   
  10    |    1    |    ↗    |    1    |    ↕    |    1    |   no    |   no   
  11    |    0    |    ↗    |    0   

Sin Eve: Alice y Bob obtuvieron 10 bits coincidentes de 15 (≈ 66.7 %). Esto está cerca del resultado esperado (~50 % en promedio), pero en una ejecución puntual puede variar.

Con Eve: Solo coincidieron 3 bits de 15 (20 %), mucho menos de lo normal. La intervención de Eve introdujo mediciones en bases erróneas, provocando que Bob reciba valores aleatorios y, al comparar bases con Alice, la mayoría de los bits se descartaran.

En resumen: la presencia de Eve reduce drásticamente la fracción de bits válidos, evidenciando su intento de espionaje.

### Referencias

Scarani, V., Bechmann-Pasquinucci, H., Cerf, N. J., Dušek, M., Lütkenhaus, N., & Peev, M. (2009). The security of practical quantum key distribution. Reviews of modern physics, 81(3), 1301-1350.
